In [1]:
%matplotlib inline

In [2]:
#export
from exp.nb_06 import *

In [3]:
xTraining, yTraining, xValidation, yValidation = getMnistData()
xTrainingNormalized, xValidationNormalized = normalizeVectors(xTraining, xValidation)
(xTrainingNormalized.mean(), xTrainingNormalized.std, xValidationNormalized.mean(), xValidationNormalized.std())

(tensor(-7.6999e-06),
 <function Tensor.std>,
 tensor(-7.0751e-08),
 tensor(1.0000))

In [4]:
numberOfClasses = 10
hiddenLayerOutput = 50
batchSize = 64
lossFuction = Functional.cross_entropy

In [5]:
trainingDataSet, validationDataSet = Dataset(xTrainingNormalized, yTraining), Dataset(xValidationNormalized, yValidation)

In [6]:
trainingDataLoader, validationDataLoader = createDataLoaders(trainingDataSet, validationDataSet, batchSize)

In [7]:
imageDataBunch = DataBunch(trainingDataLoader, validationDataLoader, numberOfClasses)

In [8]:
layerSizes = [8, 16, 32, 64, 64]

In [9]:
convolutionalModelSR1 = createBetterConvolutionModel(numberOfClasses, layerSizes)
convolutionalModelSR1

Sequential(
  (0): LambdaLayer()
  (1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): GeneralRectifiedLinearUnit()
  )
  (2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRectifiedLinearUnit()
  )
  (3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRectifiedLinearUnit()
  )
  (4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRectifiedLinearUnit()
  )
  (5): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRectifiedLinearUnit()
  )
  (6): AdaptiveAvgPool2d(output_size=1)
  (7): LambdaLayer()
  (8): Linear(in_features=64, out_features=10, bias=True)
)

In [10]:
accuracy(convolutionalModelSR1(xTraining), yTraining)

tensor(0.0994)

In [11]:
phases = [0.3, 0.7]
weightsScheduler = aggregateSchedulers(phases, createCosineSchedulers(0.3, 0.6, 0.2)) 
biasScheduler = aggregateSchedulers(phases, createCosineSchedulers(0.9, 1.8, 0.6))

In [12]:
teacher = TeacherWithHooks(schedulingFunctions=[weightsScheduler, biasScheduler])

In [13]:
teacher.teachModel(convolutionalModelSR1, imageDataBunch, 2)

Epoch #0 Training: Loss 0.4548206627368927 Accuracy 0.8388347029685974
Epoch #0 Validation: Loss 0.08740277588367462 Accuracy 0.9732001423835754

Epoch #1 Training: Loss 0.1314106285572052 Accuracy 0.9611372947692871
Epoch #1 Validation: Loss 0.14704951643943787 Accuracy 0.9635087251663208

